# Fact Produção

Tabela fato de produção com métricas de quantidade e duração.

In [ ]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {fqtn(SCHEMA_GOLD, 'fact_producao')} (
    production_sk STRING,
    equipment_id STRING,
    product_id STRING,
    start_date_key INT,
    actual_start TIMESTAMP,
    actual_end TIMESTAMP,
    duration_minutes DOUBLE,
    planned_qty INT,
    actual_qty INT,
    qty_diff INT,
    status STRING
) USING DELTA
PARTITIONED BY (start_date_key)
""")

spark.sql(f"""
INSERT OVERWRITE {fqtn(SCHEMA_GOLD, 'fact_producao')}
SELECT 
    production_order_id as production_sk,
    equipment_id,
    product_id,
    cast(date_format(actual_start, 'yyyyMMdd') as int) as start_date_key,
    actual_start,
    actual_end,
    (unix_timestamp(actual_end) - unix_timestamp(actual_start))/60 as duration_minutes,
    planned_quantity as planned_qty,
    actual_quantity as actual_qty,
    (actual_quantity - planned_quantity) as qty_diff,
    status
FROM {fqtn(SCHEMA_SILVER, 'production_orders_clean')}
WHERE actual_start IS NOT NULL
""")